In [1]:
import pandas as pd
import numpy as np

## Esports

In [2]:
csgo = pd.read_csv("./csgo_matches.csv")
csgo["match_date"] = pd.to_datetime(csgo["match_date"])
# replace win with 1 and loss with 0
csgo["win"] = csgo["win"].replace({"win": 1, "loss": -1, "draw": 0})
csgo["season"] = csgo["match_date"].dt.year

lol = pd.read_csv("./lol_matches.csv")
lol["match_date"] = pd.to_datetime(lol["match_date"])
lol["win"] = lol["win"].replace({"win": 1, "loss": -1, "draw": 0})
lol["season"] = lol["match_date"].dt.year

dota2 = pd.read_csv("./dota2_matches.csv")
dota2["match_date"] = pd.to_datetime(dota2["match_date"])
dota2["win"] = dota2["win"].replace({"win": 1, "loss": -1, "draw": 0})
dota2["season"] = dota2["match_date"].dt.year

In [3]:
# different tournaments
csgo["tournament"].value_counts()

Svenska Elitserien Spring 2022: Online Stage | Season | Group Stage      728
Svenska Elitserien Fall 2021 | Main Event | Group Stage                  712
Dust2 dk Ligaen Season 19 | Main Event | Regular Season                  530
Dust2 dk Ligaen Season 20 | Main Event | Regular Season                  522
ESEA Season 27: Premier Division - Europe | Group Stage | Group Stage    428
                                                                        ... 
WESG 2019 | Regional Finals | Russia Finals                                2
UTAGE Japan League Season 4                                                2
ESL ANZ Champs Season 10 | Open Qualifier | Qualifier #2                   2
HardCup 2020 Cup #6                                                        2
WESG 2016 Poland                                                           2
Name: tournament, Length: 11021, dtype: int64

In [4]:
# different teams
csgo["team"].value_counts()

NiP         1212
AGO         1166
aTTaX       1091
Sprout      1084
Tricked     1015
            ... 
Lol Yeah       1
sk9            1
RAFN           1
Delirium       1
Hashtag        1
Name: team, Length: 8200, dtype: int64

In [5]:
# have a look at one specific tournament
Svenska_2022 = csgo[
    csgo["tournament"]
    == "Svenska Elitserien Spring 2022: Online Stage | Season | Group Stage"
]
Svenska_2022[Svenska_2022["team"] == "X-Gamers"]["win"].value_counts()

 1    28
-1    26
Name: win, dtype: int64

In [6]:
def compute_team_metrics(game_data, min_matches=5):
    game_data = (
        game_data.groupby(["team", "tournament", "season"])["win"]
        .apply(list)
        .reset_index()
    )
    # do some calculations
    game_data["count"] = game_data["win"].apply(lambda x: len(x))
    # remove entries with less than 'min_matches' matches
    game_data = game_data[game_data["count"] > min_matches - 1]
    game_data["mean"] = game_data["win"].apply(lambda x: np.mean(x))
    game_data["var"] = game_data["win"].apply(lambda x: np.var(x))
    game_data["std"] = game_data["win"].apply(lambda x: np.std(x))
    game_data["min"] = game_data["win"].apply(lambda x: np.min(x))
    game_data["max"] = game_data["win"].apply(lambda x: np.max(x))
    game_data["median"] = game_data["win"].apply(lambda x: np.median(x))
    game_data["sum"] = game_data["win"].apply(lambda x: np.sum(x))
    return game_data

In [7]:
csgo_team_metrics = compute_team_metrics(csgo)
lol_team_metrics = compute_team_metrics(lol)
dota2_team_metrics = compute_team_metrics(dota2)

In [8]:
display(csgo_team_metrics.head())
csgo_team_metrics.shape

,team,tournament,season,win,count,mean,var,std,min,max,median,sum
4,-2.5shaa,5E Arena Open Season 3 | East Asia | Playoffs,2021,"[-1, 1, 1, -1, 1, 1]",6,0.333333,0.888889,0.942809,-1,1,1.0,2
6,-72C,RESF Open Cup 2021 | Online Stage | Group B,2021,"[-1, -1, -1, -1, -1, -1]",6,-1.000000,0.000000,0.000000,-1,-1,-1.0,-6
14,.Iceberg.,ESEA Season 26: Premier Division - North Ameri...,2017,"[-1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -...",16,0.000000,1.000000,1.000000,-1,1,0.0,0
15,.Iceberg.,ESEA Season 27: Premier Division - North Ameri...,2018,"[1, 1, 1, 1, 1, 1, -1, -1, -1, -1]",10,0.200000,0.960000,0.979796,-1,1,1.0,2
21,.Russia,World Electronic Sports Games 2016 | Group Sta...,2017,"[1, 0, -1, 1, 0]",5,0.200000,0.560000,0.748331,-1,1,0.0,1


(11453, 12)

In [9]:
lol_team_metrics.head()

,team,tournament,season,win,count,mean,var,std,min,max,median,sum
1,00Nation,Telialigaen 2021 Autumn | Main Event | Group S...,2021,"[1, 1, 1, 1, 1, 1, 1]",7,1.000000,0.000000,0.000000,1,1,1.0,7
3,06,Baltic Masters 2020 | Main Event | Group Stage,2020,"[1, 1, -1, 1, -1, 1]",6,0.333333,0.888889,0.942809,-1,1,1.0,2
5,1. BEC e.V,Summoner's Inn League 2nd Div Season 2 | Group...,2019,"[-1, 0, -1, 1, 1, -1, -1, 0]",8,-0.250000,0.687500,0.829156,-1,1,-0.5,-2
7,100,LCS 2022 Spring | Group Stage | Double Round R...,2022,"[-1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1...",18,0.333333,0.888889,0.942809,-1,1,1.0,6
9,100 AC,2018 NA Academy League Spring Split | Group St...,2018,"[-1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, 1, 1,...",18,-0.222222,0.950617,0.974996,-1,1,-1.0,-4


In [10]:
dota2_team_metrics.head()

,team,tournament,season,win,count,mean,var,std,min,max,median,sum
6,!Rebels!,NXTGAME Dota 2 Invitational 2,2016,"[1, 1, 1, 1, -1, 1]",6,0.666667,0.555556,0.745356,-1,1,1.0,4
11,!Rebels!,ProDota Cup #5,2016,"[1, 1, 1, 1, 1]",5,1.000000,0.000000,0.000000,1,1,1.0,5
17,!Rebels!,Solid Dota2 Challenge,2016,"[1, 1, 1, 1, 1, -1]",6,0.666667,0.555556,0.745356,-1,1,1.0,4
38,(monkey),The Defense 5 - Group Stage | Stage 1 | Group B,2015,"[1, 1, 1, 1, 0]",5,0.800000,0.160000,0.400000,0,1,1.0,4
42,--,MSI Beat IT GosuCup Asia - May | Stages | I: O...,2014,"[1, 1, 1, 1, 1]",5,1.000000,0.000000,0.000000,1,1,1.0,5


In [11]:
print(f"Mean variance of CS:GO teams: {csgo_team_metrics['var'].mean()}")
print(f"Mean variance of LoL teams: {lol_team_metrics['var'].mean()}")
print(f"Mean variance of Dota2 teams: {dota2_team_metrics['var'].mean()}")

Mean variance of CS:GO teams: 0.6959782701553441
Mean variance of LoL teams: 0.7206933177259153
Mean variance of Dota2 teams: 0.625068260771238
